# Outpatient Orders Prototyping Code

Get Ready

In [1]:
var DocumentStore = require('../../qewd/node_modules/ewd-qoper8-cache/node_modules/ewd-document-store');
var thisInterface = require('../../cache');
var runRPC = require('../../ewd-vista/lib/runRPC');
var sessions = require('../../qewd/node_modules/ewd-session/');
var pharm = require('../../ewd-vista-pharmacy/');
require('../../ewd-vista/lib/mFunctions')
var instance = {};
var session = '';

{}

Open Database

In [2]:
instance.db = new thisInterface.Cache();
this.db = instance.db
instance.db.open({path: '/usr/cachesys/mgr/', namespace: 'PANORAMA'});
instance.documentStore = new DocumentStore(instance.db);
console.log(instance.documentStore.db.version());

Node.js Adaptor for Cache: Version: 1.1.131 (CM); Cache Version: 2014.1 build 775


Set-up Session

In [3]:
// Initialize session management
sessions.addTo(instance.documentStore);
var session = sessions.create('testApp');
console.log('Session ID : ' + session.id);
console.log('Session token : ' + session.token);

Session ID : 50
Session token : 8c387f5b-5a0c-42f3-b6ab-796ffea3f3a8


 Initialize symbol table management in session

In [4]:
instance.db.symbolTable = sessions.symbolTable(instance.db)

{}

Sign-on Setup

In [7]:
runRPC.call(instance, {rpcName: 'XUS SIGNON SETUP'}, session);

{"type":"ARRAY","value":["ibis","ROU","PANORAMA","/usr/cachesys/mgr/undefined","5","0","PANORAMA.VISTACORE.US","0"]}

Sign-On (doesn't work)

In [8]:
var accessCode = 'MX1234'; //process.argv[3];
var verifyCode = 'MX1234!!'; //process.argv[4];

var params = { rpcName: 'XUS AV CODE', rpcArgs: [{type: 'LITERAL', value: accessCode + ';' + verifyCode}] };
console.log(params);
runRPC.call(instance, params, session);

{"rpcName":"XUS AV CODE","rpcArgs":[{"type":"LITERAL","value":"MX1234;MX1234!!"}]}


{"type":"ARRAY","value":["0","0","0","Not a valid ACCESS CODE/VERIFY CODE pair.","0","0"]}

Set Division

In [9]:
let result = true;

let divisions = runRPC.call(instance, {rpcName: 'XUS DIVISION GET'}, session).value;
divisions.splice(0,1); // Remove array length element
divisions.forEach(function(element, index, array) { // Keep only IENs
    array[index] = element.split('^')[0];
});

if (divisions.length > 1) {
params = {
  rpcName: 'XUS DIVISION SET',
  rpcArgs: [{
    type: 'LITERAL',
    value: '`' + divisions[0]
  }]
};

runRPC.call(instance, params, session)
}

Get sites associated with current division

In [18]:
var getSitesAssociatedWithCurrentLoggedInDivision = function() {
    var results = [];
    var pharmacySite = new instance.documentStore.DocumentNode('PS', [59]);
    this.db.function({function: 'SetVar^ewdVistAUtils', arguments: ['DUZ(2)', 500]});
    duz2 = this.db.function({function: 'GetVar^ewdVistAUtils', arguments: ['DUZ(2)']}).result;

    pharmacySite.forEachChild( {range: {from: 0, to: ' '} }, function(subscript, node) {
        if (duz2 === node.$('INI').value)    results.push(subscript);
    });
    return results;
}

In [19]:
getSitesAssociatedWithCurrentLoggedInDivision()

[Array] ["2","4"]

This is a copy of the code in SUMMCL^PSOORNE1: Outpatient Order Summary

In [76]:
    var aclIX = new instance.documentStore.DocumentNode('PS',['52.41','ACL'])

    var counts = {};

    aclIX.forEachChild( (clinic, node) => 
        node.forEachChild( (fmdt, node2) =>
            node2.forEachChild( (ien, node3) => {
                var z = new instance.documentStore.DocumentNode('PS',['52.41',ien,0]).value;
                if (!z) return;
                if (!['NW','RNW','RF'].includes(z.split('^')[2])) return;  // New, Renew, Refills only. No DCs etc.
                if (!z.split('^')[12]) return; // Must have clinic. Redundant check as record won't exist w/o it.
                var hospitalLocationIEN = z.split('^')[12];
                var institutionIEN = instance.db.function({function: 'GET1^DIQ', arguments: ['44', hospitalLocationIEN, 'DIVISION:INSTITUTION FILE POINTER', 'I']}).result;
                var duz2 = this.db.function({function: 'GetVar^ewdVistAUtils', arguments: ['DUZ(2)']}).result;
                if (duz2 !== institutionIEN) return;
                var hospitalLocationName = instance.db.function({function: 'EXTERNAL^DILFD', arguments: ['52.41','1.1','',hospitalLocationIEN]}).result;
                if (counts[hospitalLocationIEN] === undefined) {
                    counts[hospitalLocationIEN] = {};
                    counts[hospitalLocationIEN].ien = hospitalLocationIEN;
                    counts[hospitalLocationIEN].name = hospitalLocationName;
                    counts[hospitalLocationIEN].count = 0;
                }
                counts[hospitalLocationIEN].count++;
            })
        )
    );

    Object.keys(counts).forEach(ien => {
        console.log(counts[ien].name, counts[ien].count);
    })

    counts;

3 NORTH GU 2
ICU/CCU 1
GEN MED 10
MENTAL HEALTH 5
GENERAL MEDICINE 10
PRIMARY CARE 8
AUDIOLOGY 3
7B 3
CHILLICOTHE 11
EMPLOYEE HEALTH 2
7A GEN MED 2
CARDIOLOGY 1435
45 CLINIC PATTERN 1
20 MINUTE 2
CECELIA'S CLINIC 1
DIABETIC 2
MIKES MENTAL CLINIC 1
Emergency Department 1
GYNECOLOGIST CLINIC 1


{"6":{"ien":"6","name":"3 NORTH GU","count":2},"8":{"ien":"8","name":"ICU/CCU","count":1},"9":{"ien":"9","name":"GEN MED","count":10},"17":{"ien":"17","name":"MENTAL HEALTH","count":5},"23":{"ien":"23","name":"GENERAL MEDICINE","count":10},"32":{"ien":"32","name":"PRIMARY CARE","count":8},"64":{"ien":"64","name":"AUDIOLOGY","count":3},"81":{"ien":"81","name":"7B","count":3},"113":{"ien":"113","name":"CHILLICOTHE","count":11},"133":{"ien":"133","name":"EMPLOYEE HEALTH","count":2},"158":{"ien":"158","name":"7A GEN MED","count":2},"195":{"ien":"195","name":"CARDIOLOGY","count":1435},"206":{"ien":"206","name":"45 CLINIC PATTERN","count":1},"240":{"ien":"240","name":"20 MINUTE","count":2},"273":{"ien":"273","name":"CECELIA'S CLINIC","count":1},"285":{"ien":"285","name":"DIABETIC","count":2},"288":{"ien":"288","name":"MIKES MENTAL CLINIC","count":1},"426":{"ien":"426","name":"Emergency Department","count":1},"935":{"ien":"935","name":"GYNECOLOGIST CLINIC","count":1}}